In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/content/drive/MyDrive/EECE 490 Project/Classification_Set/augmented_final_split_fully_balanced/train"
val_dir   = "/content/drive/MyDrive/EECE 490 Project/Classification_Set/augmented_final_split_fully_balanced/val"
test_dir  = "/content/drive/MyDrive/EECE 490 Project/Classification_Set/augmented_final_split_fully_balanced/test"

img_size = (200, 200)
batch_size = 32

# No augmentation here, images are already augmented
datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

val_gen = datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

test_gen = datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)


Found 35000 images belonging to 5 classes.
Found 5000 images belonging to 5 classes.
Found 10000 images belonging to 5 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

def build_advanced_rsg_net(input_shape=(200, 200, 3), num_classes=5):
    inputs = tf.keras.Input(shape=input_shape)

    # === RSG Block 1 ===
    x = layers.Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.001))(inputs)
    x = layers.ReLU()(x)
    x = layers.Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # === RSG Block 2 ===
    x = layers.Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # === Global Average Pooling + Dense Layers ===
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.3)(x)

    # === Output Layer ===
    if num_classes == 2:
        outputs = layers.Dense(1, activation='sigmoid')(x)
    else:
        outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model


In [ ]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger

# === ENSURE CHECKPOINT DIR EXISTS
os.makedirs("/content/drive/MyDrive/EECE 490 Project/rsgnet_checkpoints", exist_ok=True)

# === OUTPUT PATHS
checkpoint_path = "/content/drive/MyDrive/EECE 490 Project/rsgnet_checkpoints/best_model.h5"
log_path = "/content/drive/MyDrive/EECE 490 Project/rsgnet_checkpoints/training_log.csv"

# === CALLBACKS
callbacks = [
    ModelCheckpoint(filepath=checkpoint_path, save_best_only=True, monitor='val_accuracy', mode='max', verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1),
    CSVLogger(log_path)
]

# === BUILD + COMPILE
model = build_advanced_rsg_net()
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# === TRAIN
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,
    callbacks=callbacks
)


Epoch 1/30
